In [1]:
import pandas as pd
import requests
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
# Initialize Dash app
app = dash.Dash(__name__)

In [3]:
# Alpha Vantage data retrieval function
def fetch_stock_data(symbol="AAPL", api_key="BCYSQOOG6GQLA1Y4"): #FJFK0XQDQULQQ0MT or BCYSQOOG6GQLA1Y4
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    # Extract and process data similar to yfinance output
    if "Time Series (1min)" in data:
        time_series = data["Time Series (1min)"]
        df = pd.DataFrame(time_series).T  # Transpose for proper formatting
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df.columns = ["Open", "High", "Low", "Close", "Volume"]
        return df.sort_index()
    else:
        print("Error: Could not retrieve data from Alpha Vantage.")
        return None

In [4]:
# Dash app layout
app.layout = html.Div([
    html.H1("Real-Time Stock Price Dashboard"),
    dcc.Input(id="ticker-input", value="AAPL", type="text", placeholder="Enter stock symbol"),
    dcc.Graph(id="stock-graph"),
    dcc.Interval(id="interval", interval=60000, n_intervals=0)  # refresh every 60 seconds
])

In [5]:
# Callback to update graph dynamically
@app.callback(
    Output("stock-graph", "figure"),
    [Input("ticker-input", "value"), Input("interval", "n_intervals")]
)
def update_graph(ticker, n_intervals):
    data = fetch_stock_data(ticker)
    if data is None:
        return go.Figure()  # return an empty figure if data is unavailable
    
    # Create the candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=data.index,
        open=data["Open"],
        high=data["High"],
        low=data["Low"],
        close=data["Close"]
    )])
    fig.update_layout(title=f"{ticker} Stock Price", xaxis_title="Time", yaxis_title="Price")
    return fig

In [6]:
# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True)